In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn 
from torchvision import models
import torch 
import gym
from rlxai.agent.actor_critic import A2C
env = gym.make("Breakout-v0")

/opt/conda/envs/xai/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()
A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


In [62]:

def img_crop(img_arr) :
    return img_arr[55:-15,15:-15,:]
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])[:,:,np.newaxis]
def totensor(img_arr) :
    return torch.FloatTensor(img_arr.transpose((2, 0, 1))).unsqueeze(dim=0)


In [63]:
actor = models.resnet18(pretrained=True)
actor.fc = nn.Linear(512, env.action_space.n)

critic = models.resnet18(pretrained=True)
critic.fc = nn.Linear(512, 1)

In [64]:
def data_transform(x) :
    x = img_crop(x)
    x = totensor(x)
    return x

In [65]:
A2C_MODEL = A2C(actor=actor,critic=critic,lr=1e-3)

In [68]:
############################################################
rewards = []
is_terminals = []
values = []
logprobs = []
############################################################
max_steps_per_episode = 100_000
batch_size = 1000
s = env.reset()
A2C_MODEL.load_model("./")
for t in range(1, max_steps_per_episode):
    s_tensor = data_transform(s)
    act , v , logprob = A2C_MODEL.choose_action(s_tensor,inference=False)
    s,r,done,info = env.step(int(act))
    rewards.append(r)
    is_terminals.append(done)
    values.append(v)
    logprobs.append(logprob)
    if batch_size ==len(rewards) | done :
        print(len(rewards) , done , r)
        s_tensor = data_transform(s)
        _ , v , _ = A2C_MODEL.choose_action(s_tensor,inference=False)
        actor_loss,  critic_loss = A2C_MODEL.get_loss(rewards , is_terminals,values,logprobs,float(v) )
        print(actor_loss , critic_loss)
        A2C_MODEL.train(actor_loss=actor_loss,critic_loss=critic_loss)
        A2C_MODEL.save_model("./")
        ############################################################
        rewards = []
        is_terminals = []
        values = []
        logprobs = []
        ############################################################



1000 False 0.0
1000
tensor(-4.3381e-07, grad_fn=<MeanBackward0>) tensor(0.0007, grad_fn=<MeanBackward0>)
